# ROI Modelbouw

#### Imports


In [1]:
import sys
import os

# voeg hier de locatie van de ROI code toe:
folder_code = "C:\Work\Projects\P24050_ROI_voor_ROR\GitHub\P1414"

# De juiste mappen worden automatisch toegevoegd
sys.path.append(folder_code + "\Code")
sys.path.append(folder_code + "\HYDROLIB_adapted\hydrolib")
sys.path.append(folder_code + "\HYDROLIB_adapted")

from data_structures.dhydro_data import DHydroData

c:\tools\miniconda3\envs\PythonVZMBakjesmodel\envs\P1414_v2\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Opzet notebook

Met deze notebook kunnen verschillende ROI modellen worden opgebouwd. Het notebook is opgebouwd in 3 apart delen, die los van elkaar kunnen worden gedraaid. In de basis faciliteren deze onderdelen het opbouwen van een geopackage in het juiste format vanuit GIS-data, het combineren van verschillende bestaande geopackages tot één gecombineerd geopackage, en het opbouwen van een D-HYDRO model vanuit een geopackage.

Voordat deze stappen kunnen worden doorlopen, is het van belang onderstaand een aantal paden en verwijzingen te definiëren.


In [2]:
# voeg hier de locatie van de GIS input folder toe. Let op: dit is de map waar de map GIS in staat!
GIS_folder = r"P:\HL-P24050\05_Analysis\01_GIS\03_Complete_GIS_database"
os.environ['GIS_folder_path'] = GIS_folder

# voeg hier de locatie toe waar de uiteindelijke modellen worden weggeschreven:
output_folder = r"C:\Werk\Projecten\P1414_ROI\Cursus\OutputModellen"


### Opties voor modelbouw
Dit notebook is opgebouwd uit 3 aparte onderdelen:

#### 1. Geopackage opbouwen vanuit leggerdata
Dit onderdeel faciliteert het opbouwen van een geopackage op basis van ruwe GIS data. Afhankelijk van de toepassing kan hier een geopackage voor 1 specifiek gebied worden opgebouwd, of gelijk geodata worden gecombineerd.

#### 2. Geopackage opbouwen vanuit afzonderlijke geopackages
Dit onderdeel faciliteert het opbouwen van een gecombineerde geopackage, waarmee een model kan worden opgebouwd. Hiermee kan bijvoorbeeld eenvoudig een geopackage worden opgebouwd waar maar een subset van de data wordt meegenomen.

#### 3. Model bouwen vanuit een geopackage
Dit onderdeel faciliteert het opbouwen van een model vanuit één geopackage. Op basis van een apart configuratiebestand worden de instellingen voor de modelbouw hierop toegepast. 

**Aanroepen onderdelen**

De verschillende onderdelen in dit notebook kunnen afzonderlijk van elkaar worden aangeroepen en gedraaid. Let hierbij wel op dat dit in de juiste volgorde gebeurt.

___
___

### 1. Geopackage opbouwen vanuit de leggerdata
De omzetting van leggerdata naar een HyDAMO compatible geopackage gebeurt aan de hand van config bestanden. In deze bestanden staan de paden naar de juiste databestanden vermeld. Daarnaast worden per dataset afzonderlijk de bestaande attributen gematcht aan de voor HyDAMO benodige attributen. 

Naast de config bestanden voor de afzonderlijke datasets is de _config_dhydro_ van belang. Hierin worden de juiste databestanden voor de opbouw van een 2D grid gespecificeerd (AHN, ruwheid, de extent van het grid) en kunnen een aantal DHYDRO specificaties worden opgegeven.

In het config bestand _defaults_ worden alle default waarden gespecificeerd voor het geval de leggerdata daar niet in voorziet. Al deze config bestanden kunnen in code editors of tekst editors (Notepad++, kladblok, etc.) aangepast worden. 

Vul in onderstaand codeblok de juiste config bestanden in waarvan je de geodata wil samenvoegen in een geopackage. De _snapdist_ is in meters, en geeft de maximale afstand aan waarbij watergangen binnen een dataset aan elkaar worden gesnapt. Deze waarde verschilt per config bestand, omdat de achterliggende geodata niet altijd even precies aansluit. 

***Tip:*** Mocht je bepaalde databases niet willen opnemen in het geopackage, comment deze regels weg met een '#'

In [3]:
# Lijst met config bestanden
# Config bestanden die niet meegenomen dienen

# De snap afstanden (snap_dist) zijn van belang voor het aan elkaar koppelen van verschillende watergangen
config_list = [
    # {'config': r"ark_nzk_config",               'snap_dist': 10},
    # {'config': r"rijnmaasmonding_open_config",  'snap_dist': 10},
    # {'config': r"rijntakken_config",            'snap_dist': 10},
    # {'config': r"noordzee_config",              'snap_dist': 200},
    # {'config': r"markermeer_config",            'snap_dist': 10},
    # {'config': r"hhsk_config",                  'snap_dist': 10},
    # {'config': r"hdsr_config",                  'snap_dist': 20},
    # {'config': r"hhd_config",                   'snap_dist': 10},
    # {'config': r"hhr_config",                   'snap_dist': 10},
    # {'config': r"wagv_config",                  'snap_dist': 10},
    # {'config': r"ontbrekende_stuwen_config",    'snap_dist': 10},
    # {'config': r"randvoorwaarden_config",       'snap_dist': 0},
    {'config': r"tunnel_config",                'snap_dist': 1},
    # {'config': r"underpass_config",             'snap_dist': 1},
]

# Naam van model config
config_dhydro = r"combined_WBD_config"
defaults = r"defaults"

# Naam van het resulterende geopackage
gpkg_file = GIS_folder + r"\GIS\HYDAMO\Tunnel_test.gpkg"

Run het onderstaande codeblok om de config bestanden in te lezen en om te zetten naar een HYDAMO geopackage.

In [5]:
dhd = DHydroData()
for ix, config in enumerate(config_list):
    
    dhd.hydamo_from_raw_data(
        defaults=defaults, config=config['config'], branch_snap_dist=config['snap_dist'], GIS_folder=GIS_folder, dhydro_config=config_dhydro
    )
    try:
        dhd.fixed_weirs_from_raw_data(config=config['config'], defaults=defaults)
    except AttributeError:
        pass

dhd.clip_structures_by_branches(buffer=5)
dhd.hydamo_to_gpkg(output_gpkg=gpkg_file)

 ----- Working on config: Tunnel -----
137 Number of np_gdf after mapping
137 Number of np_gdf after filling
Profile af625ee1-d905-4c1d-a583-66f2ff18aaa4 on branch tunn_np_105522309 inverted. Bedlevel: 2.940771818161011, Insteekhoogte: 2 (default). Insteekhoogte set to 2 m above bedlevel
Profile 9ca3e74a-9eb5-45c4-abd6-6335f0c5aa5c on branch tunn_np_118567834 inverted. Bedlevel: 6.823760032653809, Insteekhoogte: 2 (default). Insteekhoogte set to 2 m above bedlevel
Profile aefefa67-f879-42ce-a23d-eb622c6fc893 on branch tunn_np_105526049 inverted. Bedlevel: 6.928109169006348, Insteekhoogte: 2 (default). Insteekhoogte set to 2 m above bedlevel
Profile 1028e191-9ea7-4f52-b0d2-025fc1736c14 on branch tunn_np_105529931 inverted. Bedlevel: 3.35409688949585, Insteekhoogte: 2 (default). Insteekhoogte set to 2 m above bedlevel
Profile eaacc13d-8787-44a1-978d-b02ef0edc6ec on branch tunn_np_105521212 inverted. Bedlevel: 7.56669282913208, Insteekhoogte: 2 (default). Insteekhoogte set to 2 m above be

    profielpunt progress: 100%|██████████| 136/136 [00:00<00:00, 263.94 rows/s]


Clipped 4/6 features: profielpunt
Clipped 6/6 features: waterloop


___
___

### 2. Gecombineerd geopackage opbouwen vanuit afzonderlijke geopackages
In onderstaand codeblok kunnen reeds bestaande geopackages worden gecombineerd tot een gezamenlijk geopacke. Onderdeel hiervan is ook dat hoogtelijnen uit (spoor)wegen, reliëf en noordzeekeringen worden toegevoegd.

De _snapdist_ achter de geopackage is de afstand die gehanteerd wordt om te bepalen of de uiteinden van verschillende geopackages aan elkaar worden gesnapt. Hierbij geldt dat elk nieuw geopackage wordt toegevoegd aan het reeds bestaande netwerk.

In [4]:

# Geef de naam op van het nieuwe geopackage
gpkg_file = GIS_folder + r"\GIS\HYDAMO\Combined_50m_11_februari.gpkg"

# Of selecteer welke datasets gecombineerd worden in een nieuwe geopackage met bovenstaande naam
# De snap afstanden (snap_dist) zijn van belang voor het aan elkaar koppelen van verschillende databronnen
gpkgs_list = [
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\Buitenwater_11_februari.gpkg",              'snap_dist': 10},
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\HHSK_11_februari.gpkg",                     'snap_dist': 10},
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\HDSR_11_februari.gpkg",                     'snap_dist': 35},
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\HHD_15_januari.gpkg",                       'snap_dist': 10},
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\HHR_11_februari.gpkg",                      'snap_dist': 10},
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\WAGV_14_januari.gpkg",                      'snap_dist': 10},
    {'gpkg_file': GIS_folder + r"\GIS\HYDAMO\Tunnels_50m_10_januari.gpkg",               'snap_dist': 0},
]

# Specificeer de model config en de default config
config_dhydro = r"combined_WBD_config"
defaults = r"defaults"

Run onderstaand codeblok om het geopackage te creëren. 

In [ ]:
dhd = DHydroData()
for ix, gpkg in enumerate(gpkgs_list):
    print("\n" + gpkg['gpkg_file'])

    # 2. load data
    dhd.hydamo_from_gpkg(gpkg['gpkg_file'], branch_snap_dist=gpkg['snap_dist'])

# Voeg wegen, reliëf en noordzeekeringen toe aan het gecombineerde geopackage
dhd.fixed_weirs_from_raw_data(config="wegen_config", defaults=defaults)
dhd.fixed_weirs_from_raw_data(config="relief_config", defaults=defaults, min_length = 50)
dhd.fixed_weirs_from_raw_data(config="noordzeekeringen_config", defaults=defaults)

dhd.hydamo_to_gpkg(output_gpkg=gpkg_file)
print('Combined all configs into a new geopackage: '+gpkg_file)

___
___
### 3. Gecombineerd geopackage naar DHYDRO model

Vanuit het gecombineerde geopackage kan een DHYDRO model worden opgebouwd. 

Als een 2D grid gewenst is, is er de mogelijkheid om een reeds bestaand 2D grid in te laden. Het genereren van een 2D grid kan erg lang duren (2,5 uur voor 50 m resolutie), waardoor een bestaand grid inladen significant sneller werkt. 

In onderstaand blok specificeer je de bestaande 2D grids (als beschikbaar) en het geopackage dat als basis van het model dient. Specificeer daarnaast de naam van het op te bouwen model.

In [5]:
# Model name
output_folder = output_folder + r"\Combined_V3.15_50m"

# Geopackage file to build the model from
gpkg_file = GIS_folder + r"\GIS\HYDAMO\Combined_50m_11_februari.gpkg"

# Existing meshes available
existing_meshes = {
    '500m': r"P:\HL-P24050\05_Analysis\02_Model\HHSK_500m_02_januari\dflowfm\network.nc",
    '100m': r"P:\HL-P24050\05_Analysis\02_Model\Combined_V3.4_100m\dflowfm\network.nc",
    '50m': r"P:\HL-P24050\05_Analysis\02_Model\Combined_V3.3_50m\dflowfm\network.nc",
    'None': None
}

# Specificeer het config bestand met de model instellingen
config_dhydro = r"combined_WBD_config"

Run onderstaand codeblok om het model op te bouwen met het aangewezen geopackage.

***NB***: Vergeet niet om het juiste 2D grid te specificeren, als dat gewenst is!

In [ ]:
# 1. initialize an instance of DHydamoData
dhd = DHydroData()

# 2. load data
dhd.hydamo_from_gpkg(gpkg_file)

# remove brug aas not enough information is supplied
del dhd.ddm.brug
if "brug" in dhd.features:
    dhd.features.remove("brug")

# Set all the pumps to have a capacity of 0
if "gemaal" in dhd.features:
    dhd.ddm.pomp["maximalecapaciteit"] = 0

# 3. save as dhydro model
dhd.to_dhydro(
    config=config_dhydro,
    load_mesh2d_path = existing_meshes['50m'], # Specify the right mesh
    output_folder=output_folder
    )

##### Einde modelbouw
Met deze laatste stap is de modelbouw afgerond. Kopiëer het model naar de Model_database folder om het met de SAS_run_model door te kunnen rekenen.